In [1]:
import itertools
import numpy as np
import pandas as pd
from typing import Dict, Tuple, List
import logging
from dataclasses import dataclass
import time
from data_generator import TestConfiguration, create_test_instance

In [2]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
@dataclass(frozen=True)
class DPState:
    """Immutable state representation for Dynamic Programming solution.
    
    Attributes:
        capacity: Tuple of remaining room capacities for each day
        time: Current time period in booking horizon
    """
    capacity: Tuple[int, ...]  # Using tuple for immutability
    time: int

class DynamicProgramming:
    """Exact Dynamic Programming solution for hotel room pricing optimization.
    
    This implementation follows the mathematical formulation for multiple night stays 
    where the average price over the stay determines acceptance probability:
    p_t^b = (1/L_b) * sum_{i in N_b} p_{t,i}
    """
    
    def __init__(self, instance: Dict):
        """Initialize the DP solver using the test instance generated by data_generator.
        
        Args:
            instance: Dictionary containing problem parameters and data from data_generator
        """
        # Extract parameters from instance
        self.params = instance['parameters']
        self.T = self.params.T
        self.N = self.params.N
        self.C = self.params.C
        self.price_min = self.params.price_min
        self.price_max = self.params.price_max
        
        # Extract problem data
        self.booking_classes = instance['booking_classes']
        self.arrival_probs = instance['arrival_probabilities']
        self.price_sensitivity = instance['reservation_price_params']
        
        # Define number of price levels (can be adjusted based on problem size)
        self.num_price_levels = 3  # Using 3 price levels as mentioned in the example
        
        # Pre-compute price levels for each day
        self.price_levels = np.linspace(self.price_min, self.price_max, self.num_price_levels)
        
        # Generate all possible price combinations for N days
        self.price_combinations = list(itertools.product(self.price_levels, repeat=self.N))
        logger.info(f"Generated {len(self.price_combinations)} price combinations")
        
        # Pre-compute booking class stays and lengths
        self.class_stays: Dict[Tuple[int, int], Set[int]] = {}
        self.stay_lengths: Dict[Tuple[int, int], int] = {}
        for arrival, departure in self.booking_classes:
            stay_nights = set(range(arrival - 1, departure))
            self.class_stays[(arrival, departure)] = stay_nights
            self.stay_lengths[(arrival, departure)] = departure - arrival + 1
        
        # Initialize value function and policy mappings
        self.value_function: Dict[DPState, float] = {}
        self.optimal_policy: Dict[DPState, Tuple[float, ...]] = {}
        
        logger.info(f"Initialized DP solver with T={self.T}, N={self.N}, C={self.C}")
    
    def solve(self) -> Tuple[float, Dict[DPState, Tuple[float, ...]]]:
        """Solve the dynamic programming problem.
        
        Returns:
            Tuple of (optimal_value, optimal_policy)
        """
        logger.info("Starting DP solution")
        start_state = DPState(
            capacity=tuple(self.C for _ in range(self.N)),
            time=1
        )
        
        # Initialize boundary conditions
        self._initialize_boundary_conditions()
        
        # Backward induction
        for t in range(self.T, 0, -1):
            logger.info(f"Processing time period {t}")
            capacity_vectors = self._generate_feasible_capacities()
            
            for capacity in capacity_vectors:
                state = DPState(capacity=tuple(capacity), time=t)
                if sum(capacity) == 0:  # Skip fully sold out states
                    continue
                    
                optimal_value, optimal_prices = self._compute_optimal_decision(state)
                self.value_function[state] = optimal_value
                self.optimal_policy[state] = optimal_prices
        
        optimal_value = self.value_function[start_state]
        logger.info(f"DP solution completed. Optimal value: {optimal_value:.2f}")
        
        return optimal_value, self.optimal_policy
    
    def _initialize_boundary_conditions(self):
        """Initialize boundary conditions for value function."""
        # Terminal period conditions: V(x,T+1) = 0 for all x
        for capacity in self._generate_feasible_capacities():
            terminal_state = DPState(capacity=tuple(capacity), time=self.T + 1)
            self.value_function[terminal_state] = 0.0
        
        # Zero capacity conditions: V(0,t) = 0 for all t
        zero_capacity = tuple(0 for _ in range(self.N))
        for t in range(1, self.T + 2):
            sold_out_state = DPState(capacity=zero_capacity, time=t)
            self.value_function[sold_out_state] = 0.0
    
    def _compute_optimal_decision(self, state: DPState) -> Tuple[float, Tuple[float, ...]]:
        """Compute optimal value and prices for given state.
        
        Args:
            state: Current state (capacity vector and time)
            
        Returns:
            Tuple of (optimal_value, optimal_prices)
        """
        max_value = float('-inf')
        optimal_prices = tuple(self.price_min for _ in range(self.N))
        
        # Try all possible price combinations
        for prices in self.price_combinations:
            value = self._compute_expected_value(state, prices)
            if value > max_value:
                max_value = value
                optimal_prices = prices
        
        return max_value, optimal_prices
    
    def _compute_expected_value(self, state: DPState, prices: Tuple[float, ...]) -> float:
        """Compute expected value for a state-prices pair.
        
        Args:
            state: Current state
            prices: Vector of prices for each day
            
        Returns:
            Expected value considering all possible transitions
        """
        value = 0.0
        current_probs = self.arrival_probs[state.time]
        
        # No arrival case
        no_arrival_prob = 1.0 - sum(current_probs.values())
        if no_arrival_prob > 0:
            next_state = DPState(capacity=state.capacity, time=state.time + 1)
            value += no_arrival_prob * self.value_function[next_state]
        
        # For each possible booking class arrival
        for (arrival, departure), arrival_prob in current_probs.items():
            if arrival_prob <= 0:
                continue
                
            # Check capacity availability
            stay_nights = self.class_stays[(arrival, departure)]
            has_capacity = all(state.capacity[day] > 0 for day in stay_nights)
            
            if has_capacity:
                # Calculate average price for the stay
                stay_prices = [prices[day] for day in stay_nights]
                avg_price = sum(stay_prices) / self.stay_lengths[(arrival, departure)]
                
                # Compute acceptance probability using linear demand function
                eps = self.price_sensitivity[(arrival, departure)]
                accept_prob = max(0, 1 - eps * avg_price)
                
                if accept_prob > 0:
                    # State transition for accepted booking
                    next_capacity = list(state.capacity)
                    for day in stay_nights:
                        next_capacity[day] -= 1
                        
                    next_state = DPState(
                        capacity=tuple(next_capacity),
                        time=state.time + 1
                    )
                    
                    # Compute immediate revenue (sum of daily prices) and future value
                    immediate_revenue = sum(stay_prices)
                    future_value = self.value_function[next_state]
                    
                    value += arrival_prob * accept_prob * (immediate_revenue + future_value)
                
                # Rejection case
                if accept_prob < 1:
                    reject_prob = 1 - accept_prob
                    next_state = DPState(capacity=state.capacity, time=state.time + 1)
                    value += arrival_prob * reject_prob * self.value_function[next_state]
            else:
                # No capacity - treated as rejection
                next_state = DPState(capacity=state.capacity, time=state.time + 1)
                value += arrival_prob * self.value_function[next_state]
        
        return value
    
    def _generate_feasible_capacities(self) -> List[List[int]]:
        """Generate feasible capacity vectors efficiently."""
        def recursive_generate(day: int, remaining: List[int]) -> List[List[int]]:
            if day == self.N:
                return [remaining[:]]
            
            vectors = []
            for cap in range(self.C + 1):
                remaining[day] = cap
                vectors.extend(recursive_generate(day + 1, remaining))
            return vectors
        
        return recursive_generate(0, [0] * self.N)

class StochasticApproximation:
    """Implementation of the Stochastic Approximation Algorithm."""
    
    def __init__(self, instance: Dict, learning_rate: float = 0.1, 
                 num_iterations: int = 1000):
        self.instance = instance
        self.params = instance['parameters']
        self.booking_classes = instance['booking_classes']
        self.arrival_probs = instance['arrival_probabilities']
        self.epsilon = instance['reservation_price_params']
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.prices = self._initialize_prices()
        
    def _initialize_prices(self) -> Dict[int, np.ndarray]:
        """Initialize price vectors for each time period."""
        return {t: np.full(self.params.N, 
                         (self.params.price_min + self.params.price_max) / 2)
                for t in range(1, self.params.T + 1)}
    
    def solve(self) -> Tuple[float, float]:
        """Run the SAA algorithm and return the expected revenue."""
        start_time = time.time()
        
        for iteration in range(self.num_iterations):
            if iteration % 100 == 0:
                logger.info(f"SAA iteration {iteration}")
            
            # Generate sample path
            sample_path = self._generate_sample_path()
            
            # Forward pass
            revenue, gradients = self._forward_pass(sample_path)
            
            # Update prices using gradients
            self._update_prices(gradients)
        
        # Evaluate final solution
        final_revenue = self._evaluate_solution()
        solve_time = time.time() - start_time
        
        logger.info(f"SAA solution completed in {solve_time:.2f} seconds")
        return final_revenue, solve_time
    
    def _generate_sample_path(self) -> List[Tuple]:
        """Generate a sample path of customer arrivals and reservation prices."""
        path = []
        for t in range(1, self.params.T + 1):
            # Generate arrival
            probs = self.arrival_probs[t]
            classes = list(probs.keys())
            probabilities = list(probs.values())
            
            if np.random.random() < sum(probabilities):
                booking_class = np.random.choice(len(classes), p=probabilities/sum(probabilities))
                arrival, departure = classes[booking_class]
                
                # Generate reservation price
                eps = self.epsilon[(arrival, departure)]
                max_price = 1/eps  # Price where acceptance probability becomes 0
                reservation_price = np.random.uniform(self.params.price_min, max_price)
                
                path.append((t, arrival, departure, reservation_price))
            else:
                path.append((t, None, None, None))
        
        return path
    
    def _forward_pass(self, sample_path: List[Tuple]) -> Tuple[float, Dict]:
        """Perform forward pass through the sample path and compute gradients."""
        total_revenue = 0
        gradients = {t: np.zeros(self.params.N) for t in range(1, self.params.T + 1)}
        capacity = np.full(self.params.N, self.params.C)
        
        for t, arrival, departure, reservation_price in sample_path:
            if arrival is not None:
                # Check capacity
                has_capacity = True
                for day in range(arrival - 1, departure):
                    if capacity[day] < 1:
                        has_capacity = False
                        break
                
                if has_capacity:
                    # Calculate average price for the stay
                    stay_prices = self.prices[t][arrival-1:departure]
                    avg_price = np.mean(stay_prices)
                    
                    # Check if customer accepts price
                    if reservation_price >= avg_price:
                        # Accept booking
                        revenue = (departure - arrival + 1) * avg_price
                        total_revenue += revenue
                        
                        # Update capacity
                        for day in range(arrival - 1, departure):
                            capacity[day] -= 1
                        
                        # Compute price gradients
                        for day in range(arrival - 1, departure):
                            gradients[t][day] += 1  # Simplified gradient
        
        return total_revenue, gradients
    
    def _update_prices(self, gradients: Dict):
        """Update prices using computed gradients."""
        for t in range(1, self.params.T + 1):
            self.prices[t] += self.learning_rate * gradients[t]
            # Project prices to feasible range
            self.prices[t] = np.clip(self.prices[t], 
                                   self.params.price_min, 
                                   self.params.price_max)
    
    def _evaluate_solution(self, num_samples: int = 1000) -> float:
        """Evaluate the current solution using Monte Carlo simulation."""
        total_revenue = 0
        
        for _ in range(num_samples):
            sample_path = self._generate_sample_path()
            revenue, _ = self._forward_pass(sample_path)
            total_revenue += revenue
        
        return total_revenue / num_samples

In [7]:
def run_experiment1():
    """Run Experiment 1: Solution Quality Assessment."""
    logger.info("Starting Experiment 1: Solution Quality Assessment")
    
    # Create test instance
    config = TestConfiguration()
    test_params = config.get_config(
        test_type='minimal',
        market_condition='standard',
        discretization='coarse'
    )
    
    # Override parameters for small instance
    test_params.update({
        'T': 10,  # Small booking horizon
        'N': 5,   # Small service horizon
        'C': 5    # Small capacity
    })
    
    instance = create_test_instance(
        demand_scenario='base',
        market_condition='standard',
        test_configuration=test_params,
        seed=42
    )
    
    # Solve using Dynamic Programming
    dp = DynamicProgramming(instance)
    dp_revenue, dp_time = dp.solve()
    
    # Solve using Stochastic Approximation
    saa = StochasticApproximation(instance)
    saa_revenue, saa_time = saa.solve()
    
    # Calculate gap
    gap_percentage = ((dp_revenue - saa_revenue) / dp_revenue) * 100
    
    # Print results
    logger.info("\nExperiment 1 Results:")
    logger.info(f"Dynamic Programming Revenue: ${dp_revenue:.2f}")
    logger.info(f"SAA Revenue: ${saa_revenue:.2f}")
    logger.info(f"Optimality Gap: {gap_percentage:.2f}%")
    logger.info(f"DP Solution Time: {dp_time:.2f} seconds")
    logger.info(f"SAA Solution Time: {saa_time:.2f} seconds")
    
    return {
        'dp_revenue': dp_revenue,
        'saa_revenue': saa_revenue,
        'gap_percentage': gap_percentage,
        'dp_time': dp_time,
        'saa_time': saa_time
    }

In [8]:
if __name__ == "__main__":
    results = run_experiment1()

INFO:__main__:Starting Experiment 1: Solution Quality Assessment
INFO:data_generator:Generated 15 booking classes with max LOS = 7
INFO:data_generator:Initialized DataGenerator with 15 booking classes
INFO:data_generator:
Initial Price Generation Analysis for STANDARD Market
INFO:data_generator:Initialization Strategy: MARKET_BASED
INFO:data_generator:
Overall Price Statistics:
INFO:data_generator:Average Price: $231.96
INFO:data_generator:Median Price: $230.11
INFO:data_generator:Price Range: $183.87 - $335.37
INFO:data_generator:Standard Deviation: $36.41
INFO:data_generator:
Day-of-Week Price Analysis:
INFO:data_generator:Sunday: $290.68 (±$24.79)
INFO:data_generator:Monday: $212.81 (±$16.87)
INFO:data_generator:Tuesday: $210.19 (±$20.21)
INFO:data_generator:Wednesday: $209.91 (±$18.27)
INFO:data_generator:Thursday: $236.19 (±$14.00)
/Users/aeziz-local/anaconda3/envs/hotel-dynamic-pricing/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slic

TypeError: unsupported format string passed to dict.__format__